In [3]:
"""
Policy Analysis - Propuestas de politicas publicas basadas en evidencia
Ejecuta directamente, genera documento markdown y tabla CSV
"""

import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

### Configurar rutas
BASE_DIR = Path("C:/Users/ASUS/OneDrive - Universidad del Pacífico/Tareas Data Science/Minimum-Temperature-Raster")
DATA_DIR = BASE_DIR / 'data'
TABLES_DIR = DATA_DIR / 'outputs' / 'tables'
OUTPUT_DIR = DATA_DIR / 'outputs'

print("=" * 70)
print("PASO 4: ANALISIS DE POLITICAS PUBLICAS")
print("=" * 70)

### CARGAR DATOS
print("\n[1/5] Cargando resultados de analisis...")

stats_df = pd.read_csv(TABLES_DIR / 'zonal_statistics.csv')

### Identificar distritos de alto riesgo
high_risk = stats_df[stats_df['risk_category'].isin(['High', 'Very High'])].copy()
very_high_risk = stats_df[stats_df['risk_category'] == 'Very High'].copy()

### Distritos en p10 de temperatura (mas frios)
p10_threshold = stats_df['mean'].quantile(0.10)
coldest_districts = stats_df[stats_df['mean'] <= p10_threshold].copy()

print(f"Total distritos analizados: {len(stats_df)}")
print(f"Distritos de alto/muy alto riesgo: {len(high_risk)}")
print(f"Distritos de muy alto riesgo: {len(very_high_risk)}")
print(f"Distritos en percentil 10 (mas frios): {len(coldest_districts)}")
print(f"Umbral P10: {p10_threshold:.2f} C")

### REGIONES MAS AFECTADAS
print("\n[2/5] Identificando regiones criticas...")

region_summary = high_risk.groupby('REGION').agg({
    'UBIGEO': 'count',
    'mean': 'mean',
    'min': 'min',
    'frost_risk_index': 'mean'
}).rename(columns={'UBIGEO': 'n_districts'}).sort_values('n_districts', ascending=False)

print("\nRegiones con mas distritos de alto riesgo:")
print(region_summary.head(10).to_string())

### PARAMETROS PARA COSTEO
print("\n[3/5] Calculando estimaciones de costos...")

### Poblacion estimada
avg_pop_per_district = 500
total_high_risk_pop = len(high_risk) * avg_pop_per_district

### Hogares
avg_household_size = 4
total_households = total_high_risk_pop / avg_household_size

### Escuelas y centros salud
schools_per_district = 2
total_schools = len(high_risk) * schools_per_district
health_centers = len(high_risk)

print(f"\nEstimaciones para distritos de alto riesgo:")
print(f"  Poblacion objetivo: {total_high_risk_pop:,.0f} habitantes")
print(f"  Hogares: {total_households:,.0f}")
print(f"  Escuelas: {total_schools:,.0f}")
print(f"  Centros de salud: {health_centers:,.0f}")

### PROPUESTAS
print("\n[4/5] Generando propuestas de politica publica...")

### Propuesta 1
prop1_cost_per_household = 8000
prop1_coverage = 0.60
prop1_total_cost = total_households * prop1_coverage * prop1_cost_per_household
prop1_beneficiaries = int(total_households * prop1_coverage * avg_household_size)

### Propuesta 2
prop2_cost_per_family = 1200
prop2_coverage = 0.70
prop2_total_cost = total_households * prop2_coverage * prop2_cost_per_family
prop2_beneficiaries = int(total_households * prop2_coverage * avg_household_size)

### Propuesta 3
prop3_cost_per_school = 150000
prop3_cost_per_health_center = 200000
prop3_total_cost = (total_schools * prop3_cost_per_school + 
                    health_centers * prop3_cost_per_health_center)
prop3_beneficiaries = total_high_risk_pop

### Total presupuesto
total_budget = prop1_total_cost + prop2_total_cost + prop3_total_cost

print("\nPresupuesto estimado por propuesta:")
print(f"  1. Mejoramiento termico viviendas: S/ {prop1_total_cost:,.0f}")
print(f"  2. Kits anti-heladas agricolas: S/ {prop2_total_cost:,.0f}")
print(f"  3. Infraestructura educativa/salud: S/ {prop3_total_cost:,.0f}")
print(f"\n  TOTAL PRESUPUESTO: S/ {total_budget:,.0f}")

### TABLA DE PROPUESTAS
policies_df = pd.DataFrame([
    {
        'propuesta': 'Mejoramiento Termico de Viviendas',
        'objetivo': 'Reducir incidencia de IRA/EDA en 40%',
        'poblacion_objetivo': f'{len(high_risk)} distritos alto riesgo',
        'intervencion': 'Aislamiento termico, ventanas dobles, techos mejorados',
        'costo_unitario_s': prop1_cost_per_household,
        'cobertura_pct': prop1_coverage * 100,
        'beneficiarios': prop1_beneficiaries,
        'costo_total_s': prop1_total_cost,
        'kpi': 'Reduccion 40% casos IRA/EDA en MINSA; ausentismo escolar -30%',
        'plazo_años': 3
    },
    {
        'propuesta': 'Kits Anti-Heladas Agricolas',
        'objetivo': 'Reducir perdidas agricolas por heladas en 50%',
        'poblacion_objetivo': f'{len(coldest_districts)} distritos P10 temperatura',
        'intervencion': 'Mallas anti-heladas, calendarios agricolas, cobertores',
        'costo_unitario_s': prop2_cost_per_family,
        'cobertura_pct': prop2_coverage * 100,
        'beneficiarios': prop2_beneficiaries,
        'costo_total_s': prop2_total_cost,
        'kpi': 'Perdida cosechas -50%; ingreso agricola +35%; mortalidad ganado -40%',
        'plazo_años': 2
    },
    {
        'propuesta': 'Infraestructura Educativa y Salud Resiliente',
        'objetivo': 'Garantizar continuidad servicios en periodos criticos',
        'poblacion_objetivo': f'{len(high_risk)} distritos alto riesgo',
        'intervencion': 'Calefaccion escuelas/salud, aislamiento, equipamiento',
        'costo_unitario_s': (prop3_total_cost / (total_schools + health_centers)),
        'cobertura_pct': 100,
        'beneficiarios': prop3_beneficiaries,
        'costo_total_s': prop3_total_cost,
        'kpi': 'Asistencia escolar +25%; atencion salud continua 100%; friaje 0 muertes',
        'plazo_años': 4
    }
])

policies_path = OUTPUT_DIR / 'tables' / 'policy_proposals.csv'
policies_df.to_csv(policies_path, index=False)
print(f"\nTabla guardada: {policies_path.name}")

### CREAR TABLA DE REGIONES PARA MARKDOWN
print("\n[5/5] Generando documento de politicas...")

region_table = region_summary.head(5).reset_index()
region_md = "\n| Region | Distritos | Temp Media (C) | Temp Min (C) | Indice Riesgo |\n"
region_md += "|--------|-----------|----------------|--------------|---------------|\n"
for _, row in region_table.iterrows():
    region_md += f"| {row['REGION']} | {row['n_districts']} | {row['mean']:.2f} | {row['min']:.2f} | {row['frost_risk_index']:.1f} |\n"

markdown_content = f"""# POLITICAS PUBLICAS PARA MITIGACION DE HELADAS Y FRIAJE EN PERU

## DIAGNOSTICO

### Magnitud del Problema

Basado en el analisis de temperatura minima (Tmin) a nivel distrital en Peru:

- **Total de distritos analizados:** {len(stats_df):,}
- **Distritos de alto/muy alto riesgo:** {len(high_risk):,} ({len(high_risk)/len(stats_df)*100:.1f}%)
- **Distritos de muy alto riesgo:** {len(very_high_risk):,} ({len(very_high_risk)/len(stats_df)*100:.1f}%)
- **Temperatura media minima registrada:** {stats_df['min'].min():.2f}°C
- **Poblacion estimada en riesgo:** {total_high_risk_pop:,} habitantes

### Regiones Criticas

Las regiones con mayor numero de distritos de alto riesgo son:

{region_md}

### Impactos Actuales

1. **Salud:** Incremento de infecciones respiratorias agudas (IRA) y enfermedades diarreicas (EDA) durante periodos de friaje
2. **Agricultura:** Perdidas de hasta 60% en cosechas de papa, quinua y otros cultivos andinos
3. **Ganaderia:** Mortalidad de alpacas y ovinos, especialmente crias
4. **Educacion:** Ausentismo escolar de hasta 40% en meses criticos (junio-agosto)

---

## PROPUESTAS DE POLITICA PUBLICA

### PROPUESTA 1: Programa de Mejoramiento Termico de Viviendas Rurales

**Objetivo Especifico:**
Reducir en 40% la incidencia de IRA y EDA en poblacion vulnerable de distritos de alto riesgo de heladas.

**Poblacion Objetivo:**
- {len(high_risk)} distritos clasificados como alto/muy alto riesgo
- {prop1_beneficiaries:,} beneficiarios directos (60% de hogares)
- Priorizacion: Hogares con niños menores de 5 años y adultos mayores

**Intervencion:**
- Instalacion de aislamiento termico en paredes y techos
- Reemplazo de ventanas por dobles o con sellado termico
- Mejoramiento de sistemas de calefaccion (cocinas mejoradas)
- Capacitacion en uso eficiente de energia

**Costo Estimado:**
- Costo unitario: S/ {prop1_cost_per_household:,.0f} por vivienda
- Cobertura: {prop1_coverage*100:.0f}% de hogares en distritos objetivo
- **Presupuesto total: S/ {prop1_total_cost:,.0f}**

**KPIs:**
- Reduccion de 40% en casos de IRA/EDA registrados en MINSA
- Reduccion de 30% en ausentismo escolar por causas respiratorias
- Mejora de 2.5°C promedio en temperatura interna de viviendas
- 85% de satisfaccion de beneficiarios

**Plazo de Implementacion:** 3 años

---

### PROPUESTA 2: Programa de Kits Anti-Heladas para Agricultura Familiar

**Objetivo Especifico:**
Reducir en 50% las perdidas agricolas causadas por heladas en distritos del percentil 10 de temperatura minima.

**Poblacion Objetivo:**
- {len(coldest_districts)} distritos mas frios (P10 de temperatura)
- {prop2_beneficiaries:,} familias agricolas beneficiarias
- Enfoque en productores de papa, quinua, habas y ganaderos alpaqueros

**Intervencion:**
- Distribucion de mallas anti-heladas y cobertores agricolas
- Capacitacion en tecnicas de proteccion de cultivos
- Implementacion de sistemas de alerta temprana comunitarios
- Asistencia tecnica para diversificacion de cultivos
- Kits de sanidad animal para proteccion de ganado

**Costo Estimado:**
- Costo unitario: S/ {prop2_cost_per_family:,.0f} por familia
- Cobertura: {prop2_coverage*100:.0f}% de familias agricolas
- **Presupuesto total: S/ {prop2_total_cost:,.0f}**

**KPIs:**
- Reduccion de 50% en perdida de cosechas por heladas
- Incremento de 35% en ingreso agricola familiar
- Reduccion de 40% en mortalidad de ganado (alpacas, ovinos)
- 500 promotores agropecuarios capacitados

**Plazo de Implementacion:** 2 años

---

### PROPUESTA 3: Infraestructura Educativa y de Salud Resiliente al Friaje

**Objetivo Especifico:**
Garantizar la continuidad de servicios educativos y de salud durante periodos criticos de friaje.

**Poblacion Objetivo:**
- {total_schools:,} escuelas en {len(high_risk)} distritos de alto riesgo
- {health_centers:,} centros de salud/postas medicas
- {total_high_risk_pop:,} habitantes con acceso mejorado a servicios

**Intervencion:**
- Instalacion de sistemas de calefaccion en aulas y centros de salud
- Mejoramiento de aislamiento termico de infraestructura
- Provision de uniformes escolares termicos
- Equipamiento de emergencia para centros de salud (incubadoras, calefactores)
- Capacitacion a docentes y personal de salud

**Costo Estimado:**
- Costo por escuela: S/ {prop3_cost_per_school:,.0f}
- Costo por centro de salud: S/ {prop3_cost_per_health_center:,.0f}
- **Presupuesto total: S/ {prop3_total_cost:,.0f}**

**KPIs:**
- Incremento de 25% en asistencia escolar durante meses criticos
- Atencion continua de servicios de salud (100% operatividad)
- Cero muertes por friaje en poblacion con acceso a servicios mejorados
- Reduccion de 50% en cierre temporal de escuelas

**Plazo de Implementacion:** 4 años

---

## PRESUPUESTO CONSOLIDADO Y PRIORIZACION

### Resumen Presupuestal

| Propuesta | Presupuesto (S/) | Beneficiarios | Costo per capita |
|-----------|------------------|---------------|------------------|
| 1. Mejoramiento Termico Viviendas | {prop1_total_cost:,.0f} | {prop1_beneficiaries:,} | {prop1_total_cost/prop1_beneficiaries:.0f} |
| 2. Kits Anti-Heladas Agricolas | {prop2_total_cost:,.0f} | {prop2_beneficiaries:,} | {prop2_total_cost/prop2_beneficiaries:.0f} |
| 3. Infraestructura Educativa/Salud | {prop3_total_cost:,.0f} | {prop3_beneficiaries:,} | {prop3_total_cost/prop3_beneficiaries:.0f} |
| **TOTAL** | **{total_budget:,.0f}** | **{total_high_risk_pop:,}** | **{total_budget/total_high_risk_pop:.0f}** |

### Orden de Priorizacion

1. **PRIMERA PRIORIDAD:** Propuesta 3 (Infraestructura Educativa y Salud)
2. **SEGUNDA PRIORIDAD:** Propuesta 1 (Mejoramiento Termico Viviendas)
3. **TERCERA PRIORIDAD:** Propuesta 2 (Kits Anti-Heladas Agricolas)

---

## FUENTES DE FINANCIAMIENTO

- **Gobierno Central:** Presupuesto ordinario MEF (70%)
- **Gobiernos Regionales:** Recursos del canon y FONCOMUN (15%)
- **Cooperacion Internacional:** Fondos de adaptacion climatica (10%)
- **Sector Privado:** Responsabilidad social empresarial (5%)

---

## CONCLUSION

La inversion total de **S/ {total_budget:,.0f}** representa una estrategia integral para mitigar los impactos de heladas y friaje en las poblaciones mas vulnerables del Peru.

---

**Documento generado:** {pd.Timestamp.now().strftime('%Y-%m-%d')}
**Distritos analizados:** {len(stats_df):,}
"""

markdown_path = OUTPUT_DIR / 'POLICY_PROPOSALS.md'
with open(markdown_path, 'w', encoding='utf-8') as f:
    f.write(markdown_content)

print(f"Documento guardado: {markdown_path.name}")

print("\n" + "=" * 70)
print("ANALISIS DE POLITICAS PUBLICAS COMPLETADO")
print("=" * 70)
print(f"\nPresupuesto total: S/ {total_budget:,.0f}")
print(f"Beneficiarios: {total_high_risk_pop:,} habitantes")
print(f"Distritos objetivo: {len(high_risk)}")
print(f"\nArchivos generados:")
print(f"  - {policies_path.name}")
print(f"  - {markdown_path.name}")
print("=" * 70)

PASO 4: ANALISIS DE POLITICAS PUBLICAS

[1/5] Cargando resultados de analisis...
Total distritos analizados: 1789
Distritos de alto/muy alto riesgo: 342
Distritos de muy alto riesgo: 92
Distritos en percentil 10 (mas frios): 179
Umbral P10: 1.14 C

[2/5] Identificando regiones criticas...

Regiones con mas distritos de alto riesgo:
              n_districts      mean       min  frost_risk_index
REGION                                                         
PUNO                   94  0.070617 -8.802114         71.078989
CUSCO                  50  1.570964 -6.100513         65.065004
AREQUIPA               40  0.280539 -6.064533         76.403750
LIMA                   38  1.940361 -1.778746         59.566948
JUNIN                  30  1.281419 -1.876899         59.184714
HUANCAVELICA           27  1.427008 -1.820235         59.671951
APURIMAC               19  2.694730 -1.854122         56.614996
MOQUEGUA               14  1.021885 -6.741668         86.575070
AYACUCHO                9 